In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/My Drive/수업/Info_Sys_2021/PyTorch

In [ ]:
from IPython.display import Image
Image('cnn1.jpg', width=800, height=300)

In [ ]:
from IPython.display import Image
Image('cnn2.jpg', width=800, height=300)

In [ ]:
Image(open('conv.gif','rb').read())

## 비디오 lab-10-1 시청 (1 - 11분 40초까지)

### 비디오 lab-10-1 (12분 근처) 명령프롬프트 창 따라하기

#### 예제 1

In [ ]:
import torch
import torch.nn as nn

In [ ]:
conv=nn.Conv2d(1,1,11,stride=4,padding=0)

In [ ]:
conv

In [ ]:
inputs=torch.Tensor(1,1,227,227)
inputs.shape

In [ ]:
out=conv(inputs)
out.shape

#### 예제 2

In [ ]:
conv=nn.Conv2d(1,1,7,stride=2,padding=0)
inputs=torch.Tensor(1,1,64,64)
out=conv(inputs)
out.shape

#### 예제 3

#### 예제 4

#### 예제 5

Pooling: PPT 참고

### CNN 코딩 가이드

1. 라이브러리 가져오고 (torch, torchvision, matplotlib 같은것들)
2. GPU 사용 설정 하고 random value를 위한 seed 설정!
3. 학습에 사용되는 parameter 설정!(learning_rate, training_epochs, batch_size, etc)
4. 데이터셋을 가져오고 (학습에 쓰기 편하게) loader 만들기
5. 학습 모델 만들기( class CNN(torch.nn.Module) )
6. Loss function (Criterion)을 선택하고 최적화 도구 선택(optimizer)
7. 모델 학습 및 loss check(Criterion의 output)
8. 학습된 모델의 성능을 확인한다.

### 비디오 lab-10-2 (2분 근처) 명령프롬프트 창 따라하기

In [ ]:
import torch.nn as nn

In [ ]:
inputs=torch.Tensor(1,1,28,28)

In [ ]:
inputs.shape

In [ ]:
conv1=nn.Conv2d(1,32,3,padding=1)
conv1

In [ ]:
pool=nn.MaxPool2d(2)
pool

In [ ]:
conv2=nn.Conv2d(32,64,3,padding=1)
conv2

In [ ]:
out=conv1(inputs)
out.shape

In [ ]:
out=pool(out)
out.shape

In [ ]:
out=conv2(out)
out.shape

In [ ]:
out=pool(out)
out.shape

In [ ]:
out.size(0)

In [ ]:
out=out.view(out.size(0), -1)
out.shape

In [ ]:
fc=nn.Linear(3136,10)

In [ ]:
out=fc(out)

In [ ]:
out.shape

## CNN with MNIST

In [ ]:
# Lab 11 MNIST and Convolutional Neural Network
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [ ]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [ ]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = nn.Linear(7 * 7 * 64, 10, bias=True)
        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out

In [ ]:
# instantiate CNN model
model = CNN().to(device)

In [ ]:
model

In [ ]:
pip install pytorch_model_summary

In [ ]:
import pytorch_model_summary
print(pytorch_model_summary.summary(model, torch.zeros(1, 1, 28, 28).to(device), show_input=False))

In [ ]:
print(pytorch_model_summary.summary(model, torch.zeros(1, 1, 28, 28).to(device), show_input=True))

In [ ]:
from torchsummary import summary
summary(model,(1,28,28))

In [ ]:
# define cost/loss & optimizer
criterion = nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# train my model
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

In [ ]:
# Test model and check accuracy
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

In [ ]:
X.shape

pytorch_model_summary 가 torchsummary 보다 덜 헷갈릴 것으로 판단된다.